<a href="https://colab.research.google.com/github/SemihDemir13/chatbot-project-data/blob/main/v12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1
print("Proje için gerekli tüm kütüphaneler kuruluyor...")
!pip install -q -U transformers huggingface_hub accelerate sentence-transformers torch langchain langchain-community langchain-huggingface chromadb pandas peft trl bitsandbytes datasets streamlit pyngrok
print("✅ Kurulum tamamlandı.")

In [ ]:
#2
import warnings
import pandas as pd
import torch
import re
import random
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer
from datasets import Dataset
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

warnings.filterwarnings('ignore')

# Hugging Face hesabına giriş yap (Token soracak)
login()

In [ ]:
#3
# --- ADIM 1: VERİ HAZIRLIĞI ---
print("📊 Fine-Tuning için mini veri seti hazırlanıyor...")
try:
    df_for_tuning = pd.read_csv('netflix_titles.csv').fillna('').head(100)

    def format_for_tuning(row):
        return f"Movie: {row['title']}\nType: {row['type']}\nDescription: {row['description']}"

    dataset = Dataset.from_pandas(df_for_tuning)
    dataset = dataset.map(lambda x: {"text": format_for_tuning(x)})
    print("✅ Mini eğitim seti hazırlandı.")
except Exception as e:
    print(f"❌ HATA: CSV dosyası okunurken sorun oldu. Lütfen dosyayı yükleyin. Detay: {e}")

# --- ADIM 2: MODELİ EĞİTİM MODUNDA YÜKLEME ---
print("\n🧠 Model 4-bit (Eğitim Modu) olarak yükleniyor...")
model_id = "google/gemma-2b-it"

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

# --- ADIM 3: FINE-TUNING ---
print("\n🏋️‍♂️ Fine-Tuning Başlıyor... (Bu işlem 2-3 dk sürebilir)")
peft_config = LoraConfig(r=8, lora_alpha=16, task_type="CAUSAL_LM")
training_args = TrainingArguments(output_dir="./sonuclar", per_device_train_batch_size=2, max_steps=30, logging_steps=10, learning_rate=2e-4)

trainer = SFTTrainer(model=model, train_dataset=dataset, peft_config=peft_config, args=training_args)
trainer.train()
print("✅ Fine-Tuning tamamlandı!")

# --- ADIM 4: CHATBOT PİPELİNE'I OLUŞTURMA ---
print("\n🚀 Chatbot Pipeline oluşturuluyor...")
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=250)
print("✅ Sistem kullanıma hazır!")

In [ ]:
#4
print("RAG sistemi için veri ve veritabanı hazırlanıyor...")
try:
    df = pd.read_csv('netflix_titles.csv').fillna('Bilinmiyor')

    # Konu bazlı arama için dokümanları oluştur
    dokumanlar = []
    for index, row in df.sample(5000, random_state=42).iterrows(): # 5000 örnek alalım
        metin = f"Başlık: {row['title']}\nYıl: {row['release_year']}\nTür: {row['listed_in']}\nÖzet: {row['description']}"
        dokumanlar.append(metin)

    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    vektor_veritabani = Chroma.from_texts(dokumanlar, embedding=embedding_model)

    print(f"✅ Ana DataFrame ({len(df)} satır) ve Vektör Veritabanı ({len(dokumanlar)} doküman) hazırlandı.")
except Exception as e:
    print(f"❌ HATA: Veri hazırlanırken sorun oluştu. Detay: {e}")

In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# --- SAYFA AYARLARI ---
st.set_page_config(page_title="Netflix AI Chatbotu", page_icon="🎬", layout="centered")
st.title("🎬 Netflix AI Bot")
st.markdown("*Yapay Zeka Destekli Film & Dizi Öneri Asistanı*")

# --- MODEL VE VERİ YÜKLEME ---
@st.cache_resource
def sistemi_yukle():
    print("Sistem web arayüzü için yükleniyor...")
    df = pd.read_csv('netflix_titles.csv').fillna('Bilinmiyor')
    model_id = "Qwen/Qwen1.5-4B-Chat"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)
    llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=150)
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    dokumanlar = [f"Başlık: {row['title']}\nÖzet: {row['description']}" for index, row in df.iterrows()]
    vektor_db = Chroma.from_texts(dokumanlar, embedding=embedding_model)
    print("✅ Web sistemi başarıyla yüklendi.")
    return df, llm_pipeline, vektor_db

df, llm_pipeline, vektor_veritabani = sistemi_yukle()

# --- RAG FONKSİYONU (NİHAİ VERSİYON) ---
def rag_yanit(soru):
    def temizle_yorumlar(metin):
        yasakli_ifadeler = ["I hope this", "Please note", "In this translation", "What does", "The phrase means", "English:", "Turkish:", "Translation:", "小", "Note:", "번역결과", "[", "]"]
        temiz_metin = metin
        for ifade in yasakli_ifadeler:
            if ifade in temiz_metin: temiz_metin = temiz_metin.split(ifade)[0]
        temiz_metin = re.sub(r'[\u4e00-\u9fff]+', '', temiz_metin)
        return temiz_metin.replace("。", "").strip()

    # ---- DÜZELTME 1: YIL SINIRI KONTROLÜ EKLENDİ ----
    # Fonksiyonun en başında, daha hiçbir arama yapmadan, sorudaki yılı kontrol et.
    yil_tespiti_on_kontrol = re.search(r'(\d{4})', soru)
    if yil_tespiti_on_kontrol:
        aranan_yil = int(yil_tespiti_on_kontrol.group(1))
        min_yil = int(df['release_year'].min())
        max_yil = int(df['release_year'].max())
        if aranan_yil < min_yil or aranan_yil > max_yil:
            return f"Üzgünüm, veri setimde sadece {min_yil} ile {max_yil} yılları arasındaki içerikler bulunmaktadır."

    tur_ceviri = {"korku": "Horror", "gerilim": "Thriller", "aksiyon": "Action", "komedi": "Comed", "drama": "Dram", "bilim kurgu": "Sci-Fi", "romantik": "Romantic", "anime": "Anime"}
    etkisiz_kelimeler = ["filmi", "film", "öner", "bana", "yapımı", "yönetmen", "oyuncu", "konusu", "nedir", "ne", "ile", "ilgili", "tavsiye", "et"]
    temiz_soru = soru.lower()
    for kelime in etkisiz_kelimeler: temiz_soru = temiz_soru.replace(kelime, "").strip()
    anahtar_kelimeler_listesi = temiz_soru.split()
    secilen_film_bilgisi = {}; kaynak_yontem = ""; uygun_filmler = pd.DataFrame()

    aranan_tur_eng = None
    for tr_tur, eng_tur in tur_ceviri.items():
        if tr_tur in soru.lower(): aranan_tur_eng = eng_tur; break

    is_name_search = False

    if yil_tespiti_on_kontrol: # Yukarıda zaten bulmuştuk, tekrar arama
        aranan_yil = int(yil_tespiti_on_kontrol.group(1))
        kaynak_yontem = f"YIL ({aranan_yil})"; uygun_filmler = df[df['release_year'] == aranan_yil]
        if aranan_tur_eng and not uygun_filmler.empty:
            tur_filtreli = uygun_filmler[uygun_filmler['listed_in'].str.contains(aranan_tur_eng, case=False)]
            if not tur_filtreli.empty: uygun_filmler = tur_filtreli; kaynak_yontem += f" + TÜR ({aranan_tur_eng})"
    elif aranan_tur_eng:
        kaynak_yontem = f"TÜR ({aranan_tur_eng})"; uygun_filmler = df[df['listed_in'].str.contains(aranan_tur_eng, case=False)]
    elif len(temiz_soru) > 2:
        is_name_search = True
        kaynak_yontem = f"İSİM ARAMASI ('{temiz_soru}')"
        temp_df = df.copy()
        for kelime in anahtar_kelimeler_listesi:
            temp_df = temp_df[temp_df['cast'].str.contains(kelime, case=False) | temp_df['director'].str.contains(kelime, case=False)]
        uygun_filmler = temp_df

    if not uygun_filmler.empty:
        secilen = uygun_filmler.sample(1).iloc[0]
        secilen_film_bilgisi = {"title": secilen['title'], "year": secilen['release_year'], "desc": secilen['description'], "genre": secilen['listed_in']}
    else:
        if is_name_search:
            return f"Üzgünüm, veritabanımda '{temiz_soru}' adında bir yönetmen veya oyuncu bulamadım."
        kaynak_yontem = "VEKTÖR (KONU ARAMASI)"
        docs = vektor_veritabani.similarity_search(soru, k=1)
        if docs:
            bulunan_baslik = docs[0].page_content.split('\n')[0].replace("Başlık: ", "")
            tam_bilgi = df[df['title'] == bulunan_baslik]
            if not tam_bilgi.empty:
                secilen = tam_bilgi.iloc[0]
                secilen_film_bilgisi = {"title": secilen['title'], "year": secilen['release_year'], "desc": secilen['description'], "genre": secilen['listed_in']}

    if not secilen_film_bilgisi: return "Üzgünüm, uygun bir sonuç bulamadım."

    # ---- DÜZELTME 2: DAHA DA GÜÇLENDİRİLMİŞ PROMPT ----
    ingilizce_ozet = secilen_film_bilgisi['desc']
    prompt = f"""<|im_start|>system
You are a professional translator. Your ONLY task is to translate the user's English text into natural and fluent Turkish.
RULES:
1. ONLY provide the Turkish translation.
2. DO NOT add any extra notes, comments, or characters from other languages.
3. If the English text is "Bilinmiyor", translate it as "Konu bilgisi bulunmuyor.".
<|im_end|>
<|im_start|>user
{ingilizce_ozet}
<|im_end|>
<|im_start|>assistant
"""

    try:
        sonuc = llm_pipeline(prompt, return_full_text=False, max_new_tokens=150)[0]["generated_text"]
        turkce_ozet = temizle_yorumlar(sonuc)
        if len(turkce_ozet) < 5: turkce_ozet = "Konu bilgisi alınamadı."
    except: turkce_ozet = "Konu çevrilirken bir hata oluştu."

    return (f"**Önerim:** {secilen_film_bilgisi['title']} ({secilen_film_bilgisi['year']})\n"
            f"**Tür:** {secilen_film_bilgisi['genre']}\n"
            f"**Konusu:** {turkce_ozet}")

# --- CHAT ARAYÜZÜ ---
if "messages" not in st.session_state: st.session_state.messages = []
for message in st.session_state.messages:
    with st.chat_message(message["role"]): st.markdown(message["content"])

if prompt := st.chat_input("2019 yapımı korku filmi öner..."):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"): st.markdown(prompt)
    with st.chat_message("assistant"):
        with st.spinner("Arşivde aranıyor..."):
            response = rag_yanit(prompt)
            st.markdown(response)
    st.session_state.messages.append({"role": "assistant", "content": response})

In [ ]:
from pyngrok import ngrok
from google.colab import userdata

# ÖNEMLİ: Sol menüdeki Anahtar (🔑) ikonuna tıklayıp
# 'NGROK_AUTH_TOKEN' adında bir sır oluşturduğundan ve token'ını oraya yapıştırdığından emin ol.
try:
    authtoken = userdata.get('NGROK_AUTH_TOKEN')
    ngrok.set_auth_token(authtoken)
    public_url = ngrok.connect(8501)

    print(f"✅ Web Siten Hazır! Linkin bu: {public_url}")
    print("Not: Site açıldıktan sonra modelin yüklenmesi 3-5 dakika sürebilir.")

    # Arka planda Streamlit'i başlat
    !streamlit run app.py &

except Exception as e:
    print(f"❌ HATA: ngrok token'ını Colab Secrets'e eklediğinden emin misin?")
    print(f"Lütfen sol menüdeki Anahtar (🔑) ikonuna tıkla ve 'NGROK_AUTH_TOKEN' adında bir sır oluştur.")

  Stopping...
  Stopping...
